In [3]:
import datetime as dt
from datetime import timedelta

temp_date = dt.datetime.now()
print(temp_date)
post_day = str(temp_date).split(' ')
print(post_day[0])

2021-10-12 14:06:16.035090
2021-10-12


In [1]:
def convert_post_date(post_date):
    temp_date = post_date

    if '일전' in temp_date:
        post_day_list = re.findall('\d+', temp_date)
        temp_int = int(post_day_list[0])
        post_day = today - timedelta(days=temp_int)
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    elif '시간전' in temp_date:
        post_day_list = re.findall('\d+', temp_date)
        temp_int = int(post_day_list[0])
        post_day = today - timedelta(hours=temp_int)
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    elif '분전' in temp_date:
        post_day_list = re.findall('\d+', temp_date)
        temp_int = int(post_day_list[0])
        post_day = today - timedelta(minutes=temp_int)
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    elif '초전' in temp_date:
        post_day_list = re.findall('\d+', temp_date)
        temp_int = int(post_day_list[0])
        post_day = today - timedelta(seconds=temp_int)
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    elif '조금전' in temp_date:
        temp_int = 30
        post_day = today - timedelta(seconds=temp_int)
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    elif '어제' in temp_date:
        post_day = today - timedelta(hours=24)
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    elif '오늘' in temp_date:
        post_day = datetime.datetime.now()
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    elif len(temp_date) == 19:
        post_day = datetime.datetime.strptime(temp_date, '%Y. %m. %d. %H:%M')
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)

    else:
        post_day = datetime.datetime.strptime(temp_date, '%Y.%m.%d %H:%M')
        date_hour = post_day.strftime('%Y.%m.%d %H')
        tmp_day = post_day.strftime('%Y-%m-%d %H:%M')
        tmp_date = time.mktime(datetime.datetime.strptime(tmp_day, '%Y-%m-%d %H:%M').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)
    return date, date_hour



In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
from collections import Counter


HEADERS_LIST = """Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/11.0.696.50
Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/12.0.742.91""".split('\n')
HEADERS_LIST = [x.strip() for x in HEADERS_LIST]
agent = random.choice(HEADERS_LIST)
headers = {"User-Agent": "{}".format(agent)}


# 감정분석 api 변수 저장
emo_url = 'http://172.16.114.47:80/model-engine/social_senti_keyword/'

DRIVER_PATH = '/Users/kimminho/chromedriver'

driver = webdriver.Chrome(executable_path=DRIVER_PATH)


keyword = "이재명"
startDate = "20211004140224"
endDate = "20211005140224"

for pageNo in range(1, 80):
    print(pageNo)
    target_url = f"https://search.daum.net/search?w=blog&enc=utf8&q={keyword}&f=section&SA=daumsec&period=d&p={pageNo}&f=sectionsd={startDate}&ed={endDate}"
    driver.get(target_url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    
    if soup.find("span", attrs={"class":"btn_page btn_next"}):
        print("더 이상 페이지가 없습니다.")
        break
        
    if soup.find("div", attrs={"id": "noResult"}):
        print("검색결과 없음", target_url)    
        #break

    posts = soup.find("ul", attrs={"class": "list_info mg_cont clear"}).find_all("li")
    for post in posts:
        
        
        post_link = ""
        
        post_title = post.find("a", attrs={"class": "f_link_b"}).get_text()
        post_link = post.find("a", attrs={"class": "f_link_b"})['href']


        blog_name_list = post.find_all("span", attrs={"class": "f_l"})
        blog_name = blog_name_list[0].find("a", attrs={"class": "f_nb"}).get_text()

        try:
            res = requests.get(post_link, headers=headers)
            soup = BeautifulSoup(res.text, "html.parser")
            if soup.find('div', {'class': 'content_error'}):
                print("해당 게시물이 존재하지 않습니다.")
                break

            if soup.find('div', {'id':'article-reply'}):
                print("article-reply:",post_title, post_link)
                try:
                    comment_list = [p for p in soup.find_all('div', {"class":"box-content"})]

                    for tag in comment_list:
                        user_name = tag.find('strong').text
                        body = tag.find('p').text.replace("\r\n", " ")

                        temp_date_li = tag.find('span').text
                        temp_date = temp_date_li.split(' ')[0]+' '+temp_date_li.split(' ')[1]

                        date = convert_post_date(temp_date)[0]
                        dt_hour = convert_post_date(temp_date)[1]

                        temp_id = str(post_title + str(date) + user_name)
                        temp_id = temp_id.encode('utf-8')
                        h = hashlib.new('md5')
                        h.update(temp_id)
                        _id = h.hexdigest()  

                        # 감정분석 추가
                        try:
                            emo_response = requests.post(emo_url, json={'text': [comment]}, timeout=5)
                            emotions = emo_response.json()
                            source_emo = emotions['pred_labels'][0]
                            key_rk = emotions['keyword'].get("Noun_keywords")[0]
                            keyword_rk = dict(Counter(key_rk))
                            key_ak = emotions['keyword'].get("Not_Noun_keywords")[0]
                            keyword_ak = dict(Counter(key_ak))
                        except:
                            source_emo = None
                            keyword_rk = {}
                            keyword_ak = {}
                            
                        comments_dict = {
                                        '_id': _id,
                                        'title': post_title,
                                        'keyword': keyword,
                                        'date': date, #timestamp 형식의 날짜
                                        'dt_hour': dt_hour,# %Y.%m.%d %H 형식의 날짜
                                        'body': body,
                                        'user_name': user_name,
                                        'portal': "daum",
                                        'url': post_link,
                                        'emotions':source_emo,
                                        'count_rk':keyword_rk,
                                        'count_ak':keyword_ak
                                    }

#                         print(comments_dict)
                except Exception as e:
                    print(e, "댓글 없음")

            elif soup.find("div", {"id":"cContentInfo"}):
                print(post_link)
                if soup.find("div", {"id":"commentListBlock_"}):
                    comment_list2 = [p for p in soup.find_all("div",{"class":"item-reply rp_general"})]
                    for tag in comment_list2:
                        user_name = tag.find('strong').text
                        body = tag.find('div',{'class':'cont'}).text.replace("\r\n", " ")
                        
                        temp_date_li = tag.find('li', {'class':'sDateTime'}).text
                        temp_date = temp_date_li.split(' ')[0]+' '+temp_date_li.split(' ')[1]

                        date = convert_post_date(temp_date)[0]
                        dt_hour = convert_post_date(temp_date)[1]
                        temp_id = str(post_title + str(date) + user_name)
                        temp_id = temp_id.encode('utf-8')
                        h = hashlib.new('md5')
                        h.update(temp_id)
                        _id = h.hexdigest()  

                        # 감정분석 추가
                        try:
                            emo_response = requests.post(emo_url, json={'text': [comment]}, timeout=5)
                            emotions = emo_response.json()
                            source_emo = emotions['pred_labels'][0]
                            key_rk = emotions['keyword'].get("Noun_keywords")[0]
                            keyword_rk = dict(Counter(key_rk))
                            key_ak = emotions['keyword'].get("Not_Noun_keywords")[0]
                            keyword_ak = dict(Counter(key_ak))
                        except:
                            source_emo = None
                            keyword_rk = {}
                            keyword_ak = {}
                            
                        comments_dict = {
                                        '_id': _id,
                                        'title': post_title,
                                        'keyword': keyword,
                                        'date': date, #timestamp 형식의 날짜
                                        'dt_hour': dt_hour,# %Y.%m.%d %H 형식의 날짜
                                        'body': body,
                                        'user_name': user_name,
                                        'portal': "daum",
                                        'url': post_link,
                                        'emotions':source_emo,
                                        'count_rk':keyword_rk,
                                        'count_ak':keyword_ak
                                    }
                        print(comments_dict)
            else:
                print("no 댓글")
        except Exception as e:
            print(e)


In [8]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
from collections import Counter


HEADERS_LIST = """Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/11.0.696.50
Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/12.0.742.91""".split('\n')
HEADERS_LIST = [x.strip() for x in HEADERS_LIST]
agent = random.choice(HEADERS_LIST)
headers = {"User-Agent": "{}".format(agent)}


# 감정분석 api 변수 저장
emo_url = 'http://172.16.114.47:80/model-engine/social_senti_keyword/'

DRIVER_PATH = '/Users/kimminho/chromedriver'

driver = webdriver.Chrome(executable_path=DRIVER_PATH)


keyword = "이재명"
startDate = "20211004140224"
endDate = "20211005140224"

for pageNo in range(1, 80):
    print(pageNo)
    target_url = f"https://search.daum.net/search?w=blog&enc=utf8&q={keyword}&f=section&SA=daumsec&period=d&p={pageNo}&f=sectionsd={startDate}&ed={endDate}"
    driver.get(target_url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    
    if soup.find("span", attrs={"class":"btn_page btn_next"}):
        print("더 이상 페이지가 없습니다.")
        break
        
    if soup.find("div", attrs={"id": "noResult"}):
        print("검색결과 없음", target_url)    
        #break

    posts = soup.find("ul", attrs={"class": "list_info mg_cont clear"}).find_all("li")
    for post in posts:
        
        
        post_link = ""
        
        post_title = post.find("a", attrs={"class": "f_link_b"}).get_text()
        post_link = post.find("a", attrs={"class": "f_link_b"})['href']


        blog_name_list = post.find_all("span", attrs={"class": "f_l"})
        blog_name = blog_name_list[0].find("a", attrs={"class": "f_nb"}).get_text()

        res = requests.get(post_link, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        if soup.find('div', {'class': 'content_error'}):
            print("해당 게시물이 존재하지 않습니다.")
            break

        if soup.find('div', {'id':'article-reply'}):
            print("article-reply:",post_title, post_link)
            comment_list = [p for p in soup.find_all('div', {"class":"box-content"})]

            for tag in comment_list:
                user_name = tag.find('strong').text
                temp_body = tag.find('p').text.replace("\r\n", " ")
                if temp_body == "비밀댓글입니다" or temp_body == "관리자의 승인을 기다리고 있는 댓글입니다":
                    continue
                body = temp_body.replace("\n", " ")
                print(body)
                
                        

1
article-reply: 조선일보에 박수를, 이재명 기자회견장 http://blog.daum.net/bolee591/16160939
부처님 제자는 분노의 위험을 봅니다.... 세상사,사회문제,놀이거리,연극,영화등을 보면서 마음이 휩쓸리지 않습니다... 사회현상은 사회현상일뿐 그에 끄달려가지 말고 사띠를 해야합니다...
article-reply: {9월넷째주 시사} 화천대유, 이재명과 박근혜의 결탁?!?! http://blog.daum.net/smileru/8889414
스마일루가 소식이 좀 늦은 것으로 보인다 일단 화천대유의 실소유주, 속칭 '쩐주'는 이미 밝혀졌고 당연히 이재명이 아니다 2015년 킨앤파트너즈라는 투자컨설팅회사가 자본금 5천만원쩌리 페이퍼컴퍼니에 가장 먼저 400~600억을 빌려줬고, 상환을 받지 않은 채 그대로 투자금으로 전환했다 2017년에는 200~300억을 추가로 붓기까지 했다. 이런 수백억에 달하는 현금은 어디서 나왔을까? 바로 최태원의 여동생 최기원이다 수많은 기사가 이미 나왔고 본인도 부인하지 않는다 당시 최태원이 구속 중이었던 것을 생각하면, 그리고 최기원의 SK 지분율을 생각하면 사실상 회장대리의 투자 결정이라고 생각해도 좋다 킨앤파트너즈의 핵심인사는 계속 교체되었지만 SK 출신이 맡았던 것도 좋은 방증이 된다 그래도 증거가 부족하다면 이른바 '개인투자자'들 자금을 특정금전신탁으로 중개한 것도 SK증권이고 가장 노른자 땅에는 SK건설이 지은 판교SK뷰 테라스가 들어섰다는 것도 알아두면 좋다 어쨌든 2015년 재벌회장 최장기간 구속 기록을 매일 경신하고 있던 최태원은 대통령의 특별사면이 간절했고 청와대 민정수석 곽상도가 놓은 다리로 최순실에 접근해 결국 원하던 광복절 특사를 받았다 곽상도는 그 보은으로 화천대유에 아들 곽병채를 끼워넣어 50억 뇌물을 받았고 기타 보험 차원에서 당시 정권의 수많은 법복귀족들이 한자리 차지했으며 박영수는 국정농단 미르/k-스포츠 재단 사건에서 '은안' 최태원을 불기소 처분해 보답했다 그러면 최

KeyboardInterrupt: 

In [34]:
1
article-reply: 조선일보에 박수를, 이재명 기자회견장 http://blog.daum.net/bolee591/16160939
관리자의 승인을 기다리고 있는 댓글입니다
부처님 제자는 분노의 위험을 봅니다.... 세상사,사회문제,놀이거리,연극,영화등을 보면서 마음이 휩쓸리지 않습니다... 사회현상은 사회현상일뿐 그에 끄달려가지 말고 사띠를 해야합니다...
비밀댓글입니다
관리자의 승인을 기다리고 있는 댓글입니다
비밀댓글입니다
article-reply: {9월넷째주 시사} 화천대유, 이재명과 박근혜의 결탁?!?! http://blog.daum.net/smileru/8889414
스마일루가 소식이 좀 늦은 것으로 보인다 일단 화천대유의 실소유주, 속칭 '쩐주'는 이미 밝혀졌고 당연히 이재명이 아니다 2015년 킨앤파트너즈라는 투자컨설팅회사가 자본금 5천만원쩌리 페이퍼컴퍼니에 가장 먼저 400~600억을 빌려줬고, 상환을 받지 않은 채 그대로 투자금으로 전환했다 2017년에는 200~300억을 추가로 붓기까지 했다. 이런 수백억에 달하는 현금은 어디서 나왔을까? 바로 최태원의 여동생 최기원이다 수많은 기사가 이미 나왔고 본인도 부인하지 않는다 당시 최태원이 구속 중이었던 것을 생각하면, 그리고 최기원의 SK 지분율을 생각하면 사실상 회장대리의 투자 결정이라고 생각해도 좋다 킨앤파트너즈의 핵심인사는 계속 교체되었지만 SK 출신이 맡았던 것도 좋은 방증이 된다 그래도 증거가 부족하다면 이른바 '개인투자자'들 자금을 특정금전신탁으로 중개한 것도 SK증권이고 가장 노른자 땅에는 SK건설이 지은 판교SK뷰 테라스가 들어섰다는 것도 알아두면 좋다 어쨌든 2015년 재벌회장 최장기간 구속 기록을 매일 경신하고 있던 최태원은 대통령의 특별사면이 간절했고 청와대 민정수석 곽상도가 놓은 다리로 최순실에 접근해 결국 원하던 광복절 특사를 받았다 곽상도는 그 보은으로 화천대유에 아들 곽병채를 끼워넣어 50억 뇌물을 받았고 기타 보험 차원에서 당시 정권의 수많은 법복귀족들이 한자리 차지했으며 박영수는 국정농단 미르/k-스포츠 재단 사건에서 '은안' 최태원을 불기소 처분해 보답했다 그러면 최태원은 또 박영수 딸 퇴직금으로 수십억을 얹아주며 보은하겠지? 한국 법조계의 일상적 구조적 영구적 부정부패 생태계의 실상이 이러하며 사회 정의에 가격을 매겨 사고파는 더러운 관계를 부끄러워하기는커녕 아름답고 인간적이라 여긴다  이재명? 이재명이 대권주자 급으로 떠오른 것은 탄핵 이후라는 점을 기억해야 한다 변호사 자격에 직접 인허가권자이긴 하지만 당시 시민운동가 출신 야당, 재선 시장은 저 당당한 법복귀족들의 '끕'에 맞지 않는다 박근혜 정권의 실세를 야당 시장이 무슨 수로 알고 끌어모았겠는가? 그런 커넥션이 있었다면 왜 대선 정국에 진작 폭로가 되지 않았겠는가?  물론 전임 시장 김대업처럼 뇌물에 환장한 인간이었으면 기꺼이 몇푼 쥐어주고 1조에 육박하는 개발이익을 전부 SK 최씨 가문과 그 밑에 모여든 판검사 떨거지들이 나눠먹었을 것이다 그러나 이재명은 시장실에 CCTV를 설치하고 뇌물을 싸들고 오는 사람들을 쫓아냈으며 토건세력에 특혜를 주지 않겠다는 강경한 입장을 취했다 결국 SK 일가는 수익 절반을 공공개발 형태로 시가 환수하는 타협안에 만족할 수밖에 없었다 화천대유, 천하동인 고위층은 이재명과 "결탁"하기는커녕 자기 돈을 빼앗아간 불구대천의 원수로 여긴다 이재명 그 똥고집 미치광이 때문에 황금알을 낳는 거위가 반토막이 났다고 지금도 이를 갈고 있을 것이다 그러나 재벌가의 특성상, 그리고 정황상 그룹의 비자금을 동원했을 것이기 때문에 겉으로 드러나는 것을 원하지 않고 입을 다물었던 것이다 곽상도 아들 퇴직금이 50억이라는 보도가 언제 나왔는가? 최기원이 킨앤파트너즈를 거쳐 화천대유에 400억을 부었다는 보도가 나온 다음 날이었다 이런 귀찮은 일을 피하고 싶어서 수많은 법복귀족들에게 사탕을 물려준 것인데 대선판에 말려드는 것을 막지 못한, 오히려 특검을 하자고 난리를 친 곽상도를 응징한 것이다 조선일보는 세습자본의 명을 충직히 받들어 화천대유 기사를 1면에서 5면으로 옮겼고 서서히 비중을 즐여나가고 있다 장담컨대 본선에서 화천대유는 별다른 변수가 되지 못할 것이다 대선 직전, 가장 혼란할 때 기습적으로 터뜨렸다면 모를까 (윤석열 측에서 그런 준비를 하고 있었다는 말은 있다) 너무 일찍 터져서 선후관계가 다 정리가 되어버렸으니..
말해주신 내용이 꽤나 충격적이네요. 저도 좀 더 알아보겠습니다. 감사합니다. 
화천대유는 뭔가 전개가 웃기네요. 국민의힘에서 이재명 때릴 건수다 싶어서 마구 때렸는데  정작 던진돌이 자기들에게 돌아오고 있으니ㅋㅋ  50억이라는 금액의 임팩트가 워낙 크다보니 이재명이 구체적으로 얼마 해먹었다는 내용이 나오지 않는 이상 타격은 국민의 힘으로 쏠리거 같네요
그러게말입니다. 그래도 이재명이 화천대유가 돈을 받을 수 있게 한 것 아니냐, 라는 포인트가 남아있긴 해서... 그 쪽의 의심을 이재명이 어떻게 벗어날 수 있을지가 또 포인트 아닌가 싶네요. 
화천대유는 파면 뭔가 나오긴 할텐데 이재명지사와 연결고리는 아마 찾아내기 힘들지 않을까... 싶습니다. 있었으면 진작 나왔겠죠. 지금까지도 "5천억 벌어간 개꿀사업때 성남시장이었던 사람이 아무것도 없을리 없어!"라는 정황증거 수준이라... 반대로 그 의심은 합리적 근거가 없기 때문에 벗어나기도 힘들고 결국 별 이슈 없이 비비다가 사라질 것 같습니다.
저도 같은 생각입니다. 문제는 이게 선거에 어떤 영향을 미치냐일텐데... 그건 조금 더 지켜봐야 될 것 같습니다. 여론전이 뭐 어마무시하네요. 
dd님의 말대로라면 너무 일찍 대장동을 이재명과 엮어서 이재명을 추락시키려고 시도한 인사는 법관귀족들의 응징을 받는 것입니까? 곽상도에게 그러했듯..  그.. 처음에 이 건을 들춘 인물은 누구입니까. 전 그게 매우 궁금하네요.!
경기일보에서 시작되긴 했었고, 보니 해당 기자가 나름 이재명이나 경기도에 대한 적대감이 큰 것 같더라구요. 하지만 그렇다고 해서 왜곡보도를 하진 않았다고 생각됩니다. 뭔가 문제가 있었음은 분명하니까요. 
article-reply: 펌]이재명 전기 민영화하려고함 http://blog.daum.net/na-1127/2547
한전전기를 민영화한다는 말이 아닙니다 송배전 공사를 한전이 독점하고 공사비가 비싸기 때문에 송배전공사를 민간에 주겠다는 말입니다 정확한 공약은 모르지만 지금 링크한기사는 한전민영화가 아닙니다
넘나 KTX타고 광속으로 지나가면서 봐도 민영환데요? 악질인게 IMF때 한전 민영화 얘기 나올때도 송전은 제외하고 배전 민영화였어요. 근데 이 분은 송전도 포함이네요?ㅋㅋㅋㅋㅋ 솔직히 배전은 공사건수가 많지만 진짜 쫌쫌따리고 금액과 규모는 송전이 넘사인데.. 이쪽에도  뭐 해놓은게 있으신가 싶을 정도인데.......... 예 그렇게 믿고 싶은가 본데 그냥 그렇게 사세요 본인눈에 보이는 것만 보이는 법이죠 
님같은 분들덕분에 저런 쓰레기가 나대는겁니다 민영화 맞습니다
세상사람들은 그걸 민영화라고 부르고 있답니다. 
article-reply: {9월첫째주 시사} 이재명 과반 왜? 윤석열 고발사주 등 http://blog.daum.net/smileru/8889408
박용진은 입법가로서는 실적과 전망 모두 밝지만 조직의 리더나 행정가로서는 미지수 아닌가 본선에 올라와서 김두관을 제친 것만 해도 대단한 성과이다 후보 중 가장 젊은 축에 속하니 계속 도전하면 능력을 발휘할 기회가 분명 있을 것이다  다른 한편으로는 추미애의 선전이 주목할 만하다 이재명 이낙연 구도에는 미치지 못하겠지만 캠프의 의원 한 명이 없는 안타까울 정도의 조직력으로 정세균과 3위 다툼을 하고 있다 앞으로 바람에 따라 계속 역할을 할 수 있지 않을까
박용진 후보는 물론 그렇죠. 어차피 시작인 것이니까요... 그냥 좀 더 의미있는 성과를 보여주었으면 좋았을텐데 하는 아쉬움이 있네요.   추미애의 경우는 윤석열의 반대급부 아닐까 싶긴 합니다. 향후 행보가 궁금해지네요.  
이낙연/이재명을 지지하던 사람이 자기 지지하는 사람이 떨어졌다고 국힘을 뽑는다...? 잠깐 격앙되서 그런 말을 하더라도 실제로 그런 선택을 하는 사람이 과연 얼마나 될까 싶네요  윤석열 건은.... 뭐 제대로 밝혀지긴 어렵지 않으려나요 진실이더라도 법 잘 아는 사람들이니 발뺄 구석은 어떻게든 만들어놨을듯 한데
진실 밝혀지긴 어렵겠죠. 민주당도 애매한채로 남겨두려 하지 않을까 싶어요. 
개인적으로 이재명 지사 정책은 너무 극좌라 비공감하지만 저랑 똑같은 흙수저 출신 아동학대 피해자라서 지지합니다.   노무현이나 이명박 홍준표 같은 대통령이 흙이라 하는데 자식이 마음껏 공부할 수 있게 응원하는게 무슨 흙입니까.  아버지는 노름질하고 자식 학교도 안 보내고 14살에 소년공.. 이런 사람이 되어 자식을 노후대비 보험 노예로 생각하는 막장 부모를 신성시하는 유교나치 문화 좀 뿌리뽑았으면 하네요
이재명의 정책이 그런 쪽으로도 있는지는 모르겠네요. 뭐 아마 아동학대나 관련해서 뭔가 정책을 내 놓긴 하지 않을까 싶습니다. 
2
article-reply: “단군이래 최대 5503억원 공익환수” 이재명 주장 따져보니... http://blog.daum.net/heegryu/4694
article-reply: 대장동과 이재명 http://blog.daum.net/peter0516/1574
article-reply: 화천대유, 천화동인, 성남의 뜰, 이재명 아웃! http://blog.daum.net/dlscjs47/815
article-reply: 대법 연구관들 “이재명 유죄” 냈다가… 권순일 “무죄” 주장에, 추가... http://blog.daum.net/heegryu/4790
article-reply: 대장동 게이트의 몸통은 이재명입니다!| http://blog.daum.net/yunsamolovegroup/86
article-reply: 목숨 걸고 공부한 이재명 http://blog.daum.net/bolee591/16160936
무엇에 목숨을 거는 것이 엄청 중요합니다. 돈과 권력, 가족을 위해서가 아니라 사람과 중생을 위해 몸을 바치는 부처같은 정치인이 필요한거죠. 이것도 욕심인줄 압니다만 잘 선택해야 합니다. 국운이 정해지니까요... 화이팅 대한민국. 
article-reply: 화천대유 이재명. http://blog.daum.net/choong9854205/6241
민주당놈들 은근 슬쩍 국힘당 에 책임 전가...ㅋㅋㅋㅉㅉㅉ.
3
article-reply: 귀태(鬼胎) 이재명 http://blog.daum.net/cahiker/2019
article-reply: 이재명 지사ㅡ 수박론 http://blog.daum.net/k800012/5900
article-reply: 망나니 이재명의 배경 http://blog.daum.net/psk1236/2461
article-reply: 인권변호사도 가짜였던 이재명 http://blog.daum.net/na-1127/2605
어휴 진짜 저런 인간을 지지하는 사람들 정신 챙겨라~~~~!!!특히 현 쩜.오구 당댚 이하 지도부들 창피한줄 아셔라  !!!
블로그 주인인데요 영구정지 당해서 다른곳에서 찾아뵙도록 하겠습니다 ㅠㅜ 그럼 이만...........명박그네때도 안당한걸 당해봐서 참 허탈하고 어이가 없을뿐....... 여러분그인간은 절대 아니에요 모두들 안녕히...
이 글도 곧 삭제될까요?
그냥  한  마디로     '사기꾼'
인생자체가 사기네요
청문회때 이미 검증된 이낙연으로 가야합니다 국힘이 화천대유 후속으로 다른거 또 파고있대요

SyntaxError: invalid syntax (1176034898.py, line 1)

In [21]:
print(tag.find('strong').text, tag.find('span').text, tag.find('p').text.replace('\n',''))

dd 2021.09.06 09:58


'박용진은 입법가로서는 실적과 전망 모두 밝지만 조직의 리더나 행정가로서는 미지수 아닌가본선에 올라와서 김두관을 제친 것만 해도 대단한 성과이다후보 중 가장 젊은 축에 속하니 계속 도전하면 능력을 발휘할 기회가 분명 있을 것이다다른 한편으로는 추미애의 선전이 주목할 만하다이재명 이낙연 구도에는 미치지 못하겠지만캠프의 의원 한 명이 없는 안타까울 정도의 조직력으로정세균과 3위 다툼을 하고 있다앞으로 바람에 따라 계속 역할을 할 수 있지 않을까'